# Fourier transform of a Gaussian function

In [ ]:
import numpy as np
from bokeh.io import output_notebook

from fftarray import shift_frequency, shift_position
from fftarray.fft_constraint_solver import fft_dim_from_constraints
from fftarray.backends.np_backend import NumpyTensorLib
from examples.helpers import plt_fftarray

output_notebook()

def gauss_pos(x, a, sigma):
    return (a * np.exp(-(x**2/(2.* sigma**2))))/(np.sqrt(2 * np.pi) * sigma)

def gauss_freq(f, a, sigma):
    return (a * np.exp(-(1/2)*(2*np.pi*f)**2*sigma**2))

Initialize the dimension and get the basic x and f arrays.

In [ ]:
dim = fft_dim_from_constraints("x",
        pos_middle=1.,
        pos_extent = 10.,
        freq_middle = 2.5/(2*np.pi),
        freq_extent = 20./(2*np.pi),
        loose_params=["pos_extent", "freq_extent"]
    )


x = dim.fft_array(NumpyTensorLib(), space="pos")
f = dim.fft_array(NumpyTensorLib(), space="freq")

plt_fftarray(x, data_name="FFTArray (identity in pos space)")
plt_fftarray(f, data_name="FFTArray (identity in freq space)")

In [ ]:
gauss_from_pos = gauss_pos(a = 1.2, x = x, sigma = 0.7)
gauss_from_freq = gauss_freq(a = 1.2, f = f, sigma = 0.7)

np.testing.assert_array_almost_equal(gauss_from_pos, gauss_from_freq.into(space="pos"))
np.testing.assert_array_almost_equal(gauss_from_freq, gauss_from_pos.into(space="freq"))

plt_fftarray(gauss_from_pos, data_name="Gauss (initialised with position)")
plt_fftarray(gauss_from_freq, data_name="Gauss (initialised with frequency)")

Just compute a centered Gaussian in position and frequency space.
It does not matter where it is defined, the result is the same (up to numerical inaccuracies).

In [ ]:
gauss_from_pos = shift_frequency(gauss_pos(a = 1.2, x = x, sigma = 0.7), {"x": 0.9})
gauss_from_freq = gauss_freq(a = 1.2, f = f - 0.9, sigma = 0.7)

np.testing.assert_array_almost_equal(gauss_from_pos, gauss_from_freq.into(space="pos"))
np.testing.assert_array_almost_equal(gauss_from_freq, gauss_from_pos.into(space="freq"))

plt_fftarray(gauss_from_pos, data_name="Gauss (freq. shifted via FFT)")
plt_fftarray(gauss_from_freq, data_name="Gauss (freq. shifted at init)")

Now test with a Gauss shifted in position space. The `shift_position` function multiplies a phase factor unto the `FFTArray` in frequency space to achieve a user.defined shift.

In [ ]:
gauss_from_pos = gauss_pos(a = 1.2, x = x - 0.9, sigma = 0.7)
gauss_from_freq = shift_position(gauss_freq(a = 1.2, f = f, sigma = 0.7), {"x": 0.9})

np.testing.assert_array_almost_equal(gauss_from_pos, gauss_from_freq.into(space="pos"))
np.testing.assert_array_almost_equal(gauss_from_freq, gauss_from_pos.into(space="freq"))

plt_fftarray(gauss_from_pos, data_name="Gauss (pos. shifted at init)")
plt_fftarray(gauss_from_freq, data_name="Gauss (pos. shifted via FFT)")